In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# Load necessary models and scalers
xgb_model = joblib.load("xgb_model.pkl")
lstm_model = load_model("E:/OIL_InfySpringboard/Code\lstm_model.h5")
scaler = joblib.load("feature_scaler.pkl")
scaler_preds = joblib.load("prediction_scaler.pkl")

def predict_future_prices(input_data, days_ahead=10):
    """
    Predict future oil prices for a given number of days using XGBoost + LSTM model.
    """
    future_predictions = []

    # Loop through the number of days ahead you want predictions
    for day in range(days_ahead):
        # Prepare the input data for prediction (Using previous prediction to predict the next one)
        xgb_input = input_data[["Closing Price", "Selling Price", "Volume Sold"]].values.reshape(1, -1)
        
        # Scale the input data for XGBoost
        xgb_input_scaled = scaler.transform(xgb_input)
        
        # Get XGBoost predictions (output of XGBoost is used for LSTM input)
        xgb_preds = xgb_model.predict(xgb_input_scaled).reshape(-1, 1)

        # Scale XGBoost predictions
        xgb_preds_scaled = scaler_preds.transform(xgb_preds)

        # Prepare for LSTM (Reshape for LSTM model)
        xgb_preds_scaled = np.reshape(xgb_preds_scaled, (xgb_preds_scaled.shape[0], 1, 1))

        # Get LSTM prediction
        lstm_preds = lstm_model.predict(xgb_preds_scaled)

        # Store the predicted price (unscaled)
        predicted_price = scaler_preds.inverse_transform(lstm_preds).flatten()[0]
        
        future_predictions.append(predicted_price)

        # Update the input data for the next prediction
        input_data.loc[0, "Selling Price"] = predicted_price  # Update Selling Price with predicted value
    
    return future_predictions


<>:8: SyntaxWarning: invalid escape sequence '\l'
<>:8: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Admin\AppData\Local\Temp\ipykernel_18980\259657478.py:8: SyntaxWarning: invalid escape sequence '\l'
  lstm_model = load_model("E:/OIL_InfySpringboard/Code\lstm_model.h5")


TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}